# Mapas de locaciones con eventos diferente a viaje

Estos eventos son todos los que 'tipoevento' es diferente a 4 (Viaje Normal). 
Los eventos a mapearse son:

8. Devolución 
10. Recarga de la tarjeta. 
14. Devolucion

Importar librerías que vamos a usar

In [1]:
import pandas as pd
import numpy as np


import geopandas as gpd
import contextily as cx

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

Ahora vamos a cargar los archivos de datos que vamos a utlizar

In [2]:
datos_eventos = pd.read_csv("../../data/processed/consolidado-full-sin-viajes.csv", sep = ';',low_memory=False)
print(datos_eventos.sample(10))

                           serialtarjeta           idsam   
70473   2fb66f0f8b8dc6c6409aa40cfc70f6b6  043c2d42846280  \
249418  a2123d1ecf0c53bd1563c791d23b6903  04391842846280   
125190  2fcec22a3a5cccd0a1f586707af475af  04352722b95a80   
326396  fba8d6a0e4a9ac2441a621090c290474  042034d2955d80   
52389   0b34fc32a528c719624287cfdd60d951  04132a22626c80   
231744  5145193b5e92dd657b3ae1349fd020c7  04083722846280   
158091  f7d48485516ab36e3d3c0e8e9c758d70  041e07e24e6180   
315807  64a0d69d4acb4c7457da9bcb8283f787  04043122846280   
264047  fc28710e6eac78261c4c6a572cb3e137  041736e24e6180   
279109  1268242b5cdae17ab2e82171b7581709  04331122b95a80   

                      fechahoraevento producto  montoevento   
70473   2022/10/04 06:58:38.000000000       ES         1150  \
249418  2022/04/22 18:42:28.000000000       MO         2300   
125190  2022/12/06 15:34:20.000000000       MO         3400   
326396  2022/07/19 11:47:48.000000000       MO         2300   
52389   2022/10/22 06:29

Con los datos cargados, comenzamos el trabajo de limpieza y preparación para poder procesarlo y generar un mapa de eventos.

In [3]:
datos_eventos.dropna(how='any', inplace=True) # BORRAR NaNs

datos_eventos['latitude']=datos_eventos['latitude'].astype('float')
datos_eventos['longitude']=datos_eventos['longitude'].astype('float')

datos_eventos = datos_eventos[(datos_eventos[['latitude','longitude']] != 0).all(axis=1)] # Remover ceros
datos_eventos = datos_eventos[datos_eventos['latitude'] < -25.05]  # Remover latitudes fuera de Asunción
datos_eventos = datos_eventos[datos_eventos['longitude'] < -57.2] # Remover longitudes fuera de Asunción
datos_eventos = datos_eventos[datos_eventos['latitude'] > -25.8]  # Remover latitudes fuera de Asunción
# datos_eventos = datos_eventos[datos_eventos['latitude'] > -58]  # Remover latitudes fuera de Asunción



datos_eventos["tipotransporte_cat"] = pd.cut(
    x=datos_eventos["tipotransporte"],
    bins=[0, 1,3, np.inf],
    labels=["Bus municipal interno", "Normal", "Diferencial"],
)

# Se refiere al propietario del validador donde: - 1. VMT - 2. MAS - 3. JAHA
datos_eventos["identidad_cat"] = pd.cut(
    x=datos_eventos["identidad"],
    bins=[1, 2, 3, np.inf],
    labels=["VMT", "MAS", "JAHA"],
)


print(datos_eventos.sample(10))

                           serialtarjeta           idsam   
134064  5f1a0314c77e942fa13b65c213ca234b  0409171a846280  \
342435  e043c1167e8d680d4c5530b586cc9cc5  043e1542846280   
385780  0ee48e4aa0cb9b32b9d4835e7d41dd70  04132f1aaa6380   
78818   d808a7eefda0a9a8a650408b1d66e9d6  04071522846280   
59852   4694743db34ce1d406bad3486e360a65  041c28e24e6180   
142801  02178f8daa5856e51646ce9fe8c5b066  043730d2955d80   
236902  b38048b37169dd1b67d20651347c30a6  04314222b95a80   
192182  418de6c1a34f840e2d5d2bbbdb4b9908  043b1b42846280   
117404  abf80d296fb035d2c1f859995ca5bca5  04310622b95a80   
295125  ccf0e1cf3f21d8e50b958fd747a5c20d  0412361aaa6380   

                      fechahoraevento producto  montoevento   
134064  2023/01/26 17:17:26.000000000       MO         3400  \
342435  2022/08/29 12:13:30.000000000       MO         3400   
385780  2022/09/07 13:48:02.000000000       MO         2300   
78818   2022/11/23 19:01:38.000000000       MO         3400   
59852   2022/10/14 14:58

Ahora con los datos listos comenzamos a trabajar generando los mapas.
Vamos a generar diferentes mapas con estos datos: 
- serialtarjeta
- idsam
- producto
- identidad
- idrutaestacion
- tipotransporte

In [ ]:
print("=====> Generando gráficos")

latlon_por_evento= gpd.GeoDataFrame(datos_eventos, geometry=gpd.points_from_xy(datos_eventos.longitude,datos_eventos.latitude),crs="EPSG:4326")


print("Generando gráfico por producto")
eventos_mapa_plot = latlon_por_evento.plot(legend=True,column='producto',cmap='gist_rainbow',marker='o',markersize=0.2,alpha=1)
eventos_mapa_plot.set(title='Datos para producto')
cx.add_basemap(eventos_mapa_plot, crs="EPSG:4326")
plt.savefig('./graphs/producto.png',dpi=300)
plt.close()
print("Gráfico por producto listo")


print("Generando gráfico por identidad")
eventos_mapa_plot = latlon_por_evento.plot(legend=True,column='identidad_cat',cmap='gist_rainbow',marker='o',markersize=0.2,alpha=1)
eventos_mapa_plot.set(title='Datos para identidad')
cx.add_basemap(eventos_mapa_plot, crs="EPSG:4326")
plt.savefig('./graphs/identidad.png',dpi=300)
plt.close()
print("Gráfico por identidad listo")


print("Generando gráfico por idrutaestacion")
eventos_mapa_plot = latlon_por_evento.plot(legend=False,column='idrutaestacion',cmap='gist_rainbow',marker='o',markersize=0.2,alpha=1)
eventos_mapa_plot.set(title='Datos para idrutaestacion')
cx.add_basemap(eventos_mapa_plot, crs="EPSG:4326")
plt.savefig('./graphs/idrutaestacion.png',dpi=300)
plt.close()
print("Gráfico por idrutaestacion listo")


# Tipo de transporte donde - 0. Bus municipal interno - 1. Normal - 3. Diferencial
print("Generando gráfico por tipotransporte")
eventos_mapa_plot = latlon_por_evento.plot(legend=True,column='tipotransporte_cat',cmap='gist_rainbow',marker='o',markersize=0.2,alpha=1)
eventos_mapa_plot.set(title='Datos para tipotransporte')
cx.add_basemap(eventos_mapa_plot, crs="EPSG:4326")
plt.savefig('./graphs/tipotransporte.png',dpi=300)
plt.close()
print("Gráfico por tipotransporte listo")


print("Generando gráfico por idsam")
eventos_mapa_plot = latlon_por_evento.plot(legend=False,column='idsam',cmap='gist_rainbow',marker='o',markersize=0.2,alpha=1)
eventos_mapa_plot.set(title='Datos por idsam')
cx.add_basemap(eventos_mapa_plot, crs="EPSG:4326")
plt.savefig('./graphs/idsam.png',dpi=300)
plt.close()
print("Gráfico por idsam listo")


print("Generando gráfico por serialtarjeta")
eventos_mapa_plot = latlon_por_evento.plot(legend=True,column='serialtarjeta',cmap='gist_rainbow',marker='o',markersize=0.2,alpha=1)
eventos_mapa_plot.set(title='Datos por serialtarjeta')
cx.add_basemap(eventos_mapa_plot, crs="EPSG:4326")
plt.savefig('./graphs/serialtarjeta.png',dpi=300)
plt.close()
print("Gráfico por serialtarjeta listo")

print("Gráficos generados <=====")

=====> Generando gráficos
Generando gráfico por producto
Gráfico por producto listo
Generando gráfico por identidad
Gráfico por identidad listo
Generando gráfico por idrutaestacion
Gráfico por idrutaestacion listo
Generando gráfico por tipotransporte
Gráfico por tipotransporte listo
Generando gráfico por idsam
Gráfico por idsam listo
Generando gráfico por serialtarjeta
